In [1]:
# Now you can import the desired function or class
import import_ipynb
import heapq
import math
import eight_puzzle_node as EightPuzzle

In [ ]:
# A*
# Always finds optimal path
# Slower, requires more memory
# f(n)=g(n)+h(n)

# Heuristic Function
# 1. Manhattan Distance: Only straight moves allowed
# 2. Euclidean Distance: If cost is truly based on distance (e.g., maps)
# 3. Diagonal Distance: If diagonal moves are allowed, Pathfinding in games (e.g., Chess, Grid-based AI, Mazes)


# 1. Manhattan Distance Heuristic
# Total cost function = Cost function + Manhattan Distance Heuristic
# f(n) = g(n) + h(n))
# Where, 
# a. (g(n)) = Cost Function 
# The cost function represents the actual number of moves taken to reach a state from the initial state.
# Here, cost = g(n), which is simply incremented by 1 for each valid move.
# b. (h(n)) = Manhattan Distance Heuristic 
# Manhattan distance calculates the sum of the absolute differences between the current position (i, j) 
# of a tile and its goal position (goal_x, goal_y).
# Formula:
# h(n) = E(|X_current - X_goal| + |Y_current - Y_goal|)


# 2. Euclidean Distance Heuristic
# (h(n)) = sqrt ( (current_cell.x – goal.x)2 + (current_cell.y – goal.y)2 )


# 3. Diagonal Distance Heuristic
# dx = abs(current_cell.x – goal.x)
# dy = abs(current_cell.y – goal.y)
# h = D * (dx + dy) + (D2 - 2 * D) * min(dx, dy)
# where D is length of each node(usually = 1) and D2 is diagonal distance between each node (usually = sqrt(2) ). 

In [11]:
class IDistance:
    def calculate(self, puzzle: EightPuzzle.EightPuzzle): raise NotImplementedError

In [4]:
# Euclidean Distance Heuristic (h(n))
class EuclideanDistance(IDistance): 
    def __init__(self):
        print("EuclideanDistance")
        pass
    
    def __findPosition(self, value, goal_state):
        for r in range(0, len(goal_state)):
            for s in range(0, len(goal_state[r])):
                if goal_state[r][s] == value:
                    return r, s
        
    def calculate(self, puzzle: EightPuzzle.EightPuzzle):
        """Computes the Manhattan Distance heuristic."""
        distance = 0
        
        state = puzzle.state
        goal_state = puzzle.goal_state
        
        for x in range(0, len(state)):
            for y in range(0, len(state[x])):
                value = state[x][y]
                if value != 0: # Ignore empty tile
                    goal_x, goal_y = self.__findPosition(value, goal_state)
                    v = abs(x - goal_x)/2 + abs(y - goal_y)/2
                    ed = math.sqrt(v)
                    distance = distance + ed
        return distance

In [5]:
# Diagonal Distance Heuristic (h(n))
class DiagonalDistance(IDistance): 
    def __init__(self):
        print("DiagonalDistance")
        pass
    
    def __findPosition(self, value, goal_state):
        for r in range(0, len(goal_state)):
            for s in range(0, len(goal_state[r])):
                if goal_state[r][s] == value:
                    return r, s
        
    def calculate(self, puzzle: EightPuzzle.EightPuzzle):
        """Computes the Manhattan Distance heuristic."""
        distance = 0
        
        state = puzzle.state
        goal_state = puzzle.goal_state
        
        for x in range(0, len(state)):
            for y in range(0, len(state[x])):
                value = state[x][y]
                if value != 0: # Ignore empty tile
                    goal_x, goal_y = self.__findPosition(value, goal_state)
                    dx = abs(x - goal_x)
                    dy = abs(y - goal_y)
                    d = 1
                    d2 = math.sqrt(2)
                    m = min(dx, dy)
                    dd = d * (dx + dy) + (d2 - 2 * d) * m
                    distance = distance + dd
        return distance

In [6]:
# Manhattan Distance Heuristic (h(n))
class ManhattanDistance(IDistance): 
    def __init__(self):
        print("ManhattanDistance")
        pass
        
    def __findPosition(self, value, goal_state):
        for r in range(0, len(goal_state)):
            for s in range(0, len(goal_state[r])):
                if goal_state[r][s] == value:
                    return r, s
        
    def calculate(self, puzzle: EightPuzzle.EightPuzzle):
        """Computes the Manhattan Distance heuristic."""
        distance = 0
        
        state = puzzle.state
        goal_state = puzzle.goal_state
        
        for x in range(0, len(state)):
            for y in range(0, len(state[x])):
                value = state[x][y]
                if value != 0: # Ignore empty tile
                    goal_x, goal_y = self.__findPosition(value, goal_state)
                    md = abs(x - goal_x) + abs(y - goal_y)
                    distance = distance + md
        return distance
        

In [7]:
# Cost function (g(n)) - Move count
class Cost:
    __path_length: int = 0
    
    def __init__(self):
        self.value = 0
    
    def update(self):
        """Cost function: Number of moves taken to reach the current state."""
        self.__path_length = self.__path_length + 1
    
    def get(self):
        return self.__path_length


In [9]:
# Create A*
class AstarSearch:
    puzzle: EightPuzzle.EightPuzzle = None
    cost: Cost = None
    distance: IDistance = None
    
    def __init__(self, puzzle: EightPuzzle.EightPuzzle, cost: Cost, distance: IDistance):
        self.puzzle = puzzle
        self.cost = cost
        self.distance = distance
        print("AstarSearch")
    
    # def __init__(self, puzzle: EightPuzzle.EightPuzzle):
    #     self.puzzle = puzzle
    
    def solve(self):
        visited = set()
        priority_queue = [] 
        
        g_n = self.cost.get()  # Initial cost (0 moves)
        h_n = self.distance.calculate(self.puzzle)  # Initial heuristic
        start_node = (g_n + h_n, g_n, self.puzzle, [])  # (f(n), g(n), puzzle, path)
        heapq.heappush(priority_queue, start_node)
    
        while priority_queue:
            _, g_n, current, path = heapq.heappop(priority_queue)
            state_tuple = tuple(map(tuple, current.state))
            
            if state_tuple in visited:
                continue
            
            visited.add(state_tuple)
            current.display_state()
            print("---------------")
            
            if current.is_goal_reached():
                print("Goal state reached!")
                print("Solution path:", path)
                return
            
            
            lists = current.get_possible_moves() 
            print(lists)
            for move in lists:
                next_state: EightPuzzle = current.move(move)
                if next_state:
                    self.cost.update()  # Increase cost g(n)
                    new_g_n = self.cost.get()
                    new_h_n = self.distance.calculate(next_state)
                    next_node = (new_g_n + new_h_n, new_g_n, next_state, path + [move])
                    heapq.heappush(priority_queue, next_node)
        
        print("No solution found")
        return None